In [261]:
from os.path import join, basename, splitext
import json
from glob import glob
from tqdm import tqdm
from itertools import islice
import numpy as np

from common import tokenize
import logging

from IPython.display import Markdown, display, HTML
from common import get_all_docs

logger = logging.getLogger(__name__)
DATA_FOLDER = '../data/'

In [60]:
all_docs = get_all_docs(DATA_FOLDER)
val_docs = glob(join(DATA_FOLDER, 'validate/*.txt'))

with open(join(DATA_FOLDER, 'best.json'), 'r') as f:
    best = json.load(f)
    
with open(join(DATA_FOLDER, 'gold.json'), 'r') as f:
    gold = json.load(f)

In [120]:
def hprint(string, i=1):
    display(HTML('<h%s>%s</h%s>' % (i, string, i)))

def print_file(sub_name, domain, i=1, tokenize=False):    
    found = [fn for fn in domain if sub_name in fn]
    for fn in found:
        hprint(fn + '\n', i)
        with open(fn, 'r') as f:
            text = f.read()
            if tokenize:
                print(tokenize(text))
            else:
                print(text)
#         !cat {'\''+fn+'\''}

def inspect_result(qname, start_index=0, end_index=2):
    true_docs = gold[qname]

    hprint('Query\n')
    print_file(qname, val_docs, 5)
    for i, tname in enumerate(true_docs):
        hprint('True %s\n' %(i+1))
        print_file(tname, all_docs, 5)

    for i, bname in enumerate(best[qname][start_index:end_index]):
        hprint('Found %s\n' % (i+1))
        print_file(bname, all_docs, 5)    

In [115]:
val_docs_ids = set([splitext(basename(f))[0] for f in val_docs])
validation_ids = [k for k in gold.keys() if k in val_docs_ids]
print('validation set size %s' % len(validation_ids))
for qname in validation_ids:
    print(qname)

validation set size 130
_40_1a_2011126928A10010101RU
_df_5c_2010114200A10010101RU
_ab_68_2006139128A10010101RU
_1b_60_2010122997A10010101RU
_ef_e0_2010102239A10010101RU
_a5_03_2009140669A10010101RU
_87_7b_2010120096A10010101RU
_c3_b8_2009111341A10010101RU
_ff_96_2010138984A10010101RU
_be_ef_2010152978A10010101RU
_59_59_2009131618A10010101RU
_87_6d_2008148713A10010101RU
_0c_89_2009110686A10010101RU
_03_43_2008151996A10010101RU
_94_b1_2009122096A10010101RU
_e6_cc_2009147572A10010101RU
_18_b9_2010101690A10010101RU
_79_fe_2010107007A10010101RU
_ae_fc_2010107173A10010101RU
_8d_51_2010136608A10010101RU
_05_7a_2007148915A10010101RU
_82_1b_2010112743A10010101RU
_91_ca_2010129617A10010101RU
_3f_fb_2010139538A10010101RU
_23_fc_2011105696A10010101RU
_b6_9c_2010104075A10010101RU
_f0_fa_2010100979A10010101RU
_14_9d_2011108876A10010101RU
_a8_27_2010114657A10010101RU
_c9_ad_2010109635A10010101RU
_c0_79_2009148052A10010101RU
_33_40_2009146768A10010101RU
_88_8b_2011124782A10010101RU
_8f_ca_2011124783A1

# Характеристика качества данных

| Заявка | Порочащий д-т без описания   | Хорошее совпадение назаний|
|------|------|
|   _40_1a_2011126928A10010101RU  | 1||
|   _df_5c_2010114200A10010101RU  | 1|1|

# Подход

In [131]:
import gensim

model = gensim.models.Word2Vec.load(join(DATA_FOLDER, 'vectors/old/w2v_model_300'))
wv = model.wv

In [217]:
wv.most_similar(positive=['суть'], topn=10)

[('принцип', 0.6708181500434875),
 ('рассматривать', 0.6258403062820435),
 ('идея', 0.620353639125824),
 ('излагать', 0.6177359223365784),
 ('пояснить', 0.6016724109649658),
 ('сводиться', 0.5885085463523865),
 ('очевидный', 0.5867215394973755),
 ('укрупнить', 0.5856279134750366),
 ('принципиальный', 0.5842350721359253),
 ('чертеж', 0.5810674428939819)]

In [233]:
def most_probable(words, sent, n=4):
    if len(words) <= n:
        return words

    logger = logging.getLogger(__name__)
    wv.init_sims()

    used_words = set([word for word in words if word in wv])
    if len(used_words) != len(words):
        ignored_words = set(words) - set(used_words)
        logger.warning("vectors for words %s are not present in the model, ignoring these words", ignored_words)
    if not used_words:
        raise ValueError("cannot select a word from an empty list")
    vectors = np.vstack(wv.word_vec(word, use_norm=True) for word in used_words).astype(np.float32)
    mean = gensim.matutils.unitvec(vectors.mean(axis=0)).astype(np.float32)
    dists = np.dot(vectors, mean)
    most = sorted(zip(dists, used_words), reverse=True)[:n]
    
    return [word for sim, word in most]

In [235]:
sent = 'РЕФЕРАТ описывать способ получение ВНУТРЕННИЙ ГИДРОФТОРОЛЕФИН формула'
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

TypeError: object of type 'KeyedVectors' has no len()

In [131]:
import gensim

model = gensim.models.Word2Vec.load(join(DATA_FOLDER, 'vectors/old/w2v_model_300'))
wv = model.wv

In [140]:
wv.most_similar(positive=['результат'], topn=30)

[('получать', 0.666301429271698),
 ('предлагать', 0.6384239792823792),
 ('неудачн', 0.6303017735481262),
 ('заключаться', 0.6248302459716797),
 ('изобретение', 0.597603440284729),
 ('обеспечивать', 0.584223747253418),
 ('использование', 0.5819722414016724),
 ('существенный', 0.579962432384491),
 ('достигаться', 0.5796698927879333),
 ('достигаемый', 0.5791462659835815),
 ('позволять', 0.5789086818695068),
 ('известный', 0.5778968334197998),
 ('приводить', 0.5773374438285828),
 ('создание', 0.5725580453872681),
 ('повышение', 0.5720114707946777),
 ('задача', 0.5653373003005981),
 ('эффективность', 0.5649670362472534),
 ('образ', 0.5633858442306519),
 ('новизна', 0.5626537203788757),
 ('позволить', 0.5611244440078735),
 ('возможность', 0.5591506958007812),
 ('выполнение', 0.5543525218963623),
 ('данный', 0.5468902587890625),
 ('недостаток', 0.5466387867927551),
 ('самый', 0.5442663431167603),
 ('путем', 0.5432099103927612),
 ('однако', 0.5426965355873108),
 ('решение', 0.5411698818206787)

In [265]:
def most_probable(wv, words, n=4, enable_logging=True):
    if len(words) <= n:
        return words

    wv.init_sims()
    
    used_words = set([word for word in words if word in wv])
    if len(used_words) != len(words):
        ignored_words = set(words) - set(used_words)
        if enable_logging:
            logger.warning("vectors for words %s are not present in the model, ignoring these words", ignored_words)
    if not used_words:
        raise ValueError("cannot select a word from an empty list")
    vectors = np.vstack(wv.word_vec(word, use_norm=True) for word in used_words).astype(np.float32)
    mean = gensim.matutils.unitvec(vectors.mean(axis=0)).astype(np.float32)
    dists = np.dot(vectors, mean)
    most = sorted(zip(dists, used_words), reverse=True)[:n]
    
    return [word for sim, word in most]

In [255]:
sent = 'РЕФЕРАТ описывать способ получение ВНУТРЕННИЙ ГИДРОФТОРОЛЕФИН формула'
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

final n-gram: ['описывать', 'получение', 'формула']


In [256]:
sent = 'задача решение который направлять настоящий изобретение'
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

final n-gram: ['изобретение', 'настоящий', 'задача']


In [257]:
sent = 'новый технологичный способ получение'
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

final n-gram: ['способ', 'получение', 'новый']


#### Теперь придумаем новую фразу

In [266]:
sent = 'в настоящий время автомобильный промышленность сталкиваться с предписание в отношение потенциал глобальный потепление ( gwp ) для хладагент , использовать в автомобильный кондиционер'
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

vectors for words {'(', 'gwp', 'в', ',', 'с', ')', 'для'} are not present in the model, ignoring these words


final n-gram: ['потепление', 'глобальный', 'автомобильный']


# Пример

In [127]:
inspect_result('_ab_68_2006139128A10010101RU', 0,3)

﻿способ получение фторировать олефин .
 реферат описывать способ получение внутренний гидрофторолефин формула rcй = chc2f5 , включать взаимодействие фторированный олефин формула rcй = chf , где r выбирать из перфторалкильный группа , иметь от один до десять атом углерод , и х выбирать из группа , включать н и f .
 алкильный цепь r представлять себя или линейный н - алкильный цепь , разветвлять втор - алкильный цепь , такой как cf3cf2c ( cf3 ) f - , или разветвлять изоалкильный цепь , такой как cf3cf ( cf3 ) cf2- .
 дополнительно разрабатывать способ присоединение терминальный фторировать олефин к тетрафторэтилен . .
 1 .
 способ получение дигидрофторолефин , иметь формула rch = chc2f5 , включать : взаимодействие фторированный олефин формула rch = chf , где r представлять себя перфторировать алкильный группа , иметь от один до десять атом углерод , и указанный алкильный группа иметь или н - алкильный цепь , втор - алкильный цепь , или изоалкильный цепь , в жидкий фаза с тетрафторэтилен 

способ получение полифторолефин , фтор-1-этилциклопентен и перфтор-2,3-дихлогекс-2-ен или перфтор-4,5-дихлорокт-4-ен  . 
 предмет изобретение являться способ получение полифторолефин путем каталитический присоединение полифтораллилфторидов к фторэтилен .
 продукт присоединение с соотношение 1:1 и 1:2 , такой как фторпентен-2 и фторгептен-3 , мочь образовывать селективный с последующий восстановление до дегидро- или тригидрополифторалкан , который являться полезный фторуглеводородный очищающий средство .
 2 с . и 28 з . п . ф-лы , 1 табл .  . 
 1 .
 способ получение полифторолефин , содержать по крайний мера 5 атом углерод , путем взаимодействие первое полифторолефин , содержать группа галоидный аллить , со второе полифторолефин в присутствие катализатор , отличать тем , что в качество первое полифторолефин применять олефин формула где r1 f , cl , h или rf ; r2 f , cl , h или rf ; r3 f , cl или h ;
 rf c2-c1 2 перфторалкить , возможно содержать один атом h или один атом cl ; r4 f , или 

фторировать углеводород , способ они получение , смазка для холодильный машина и смазка для магнитный регистрировать среда  . 
 описываться новый фторировать углеводород , содержать в молекула , по крайний мера , один атом кислород формула i , где каждый из r , r и r представлять себя частично или полностью фторировать линейный или разветвлять алкильный группа , содержать от 1 до 30 атом углерод , r представлять себя линейный или разветвлять алкильный группа , содержать от 1 до 30 атом углерод или простой полиэфирный группа , содержать от 2 до 100 атом углерод .
 описываться способ они получение , смазка для холодильный машина и смазка для магнитный регистрировать среда на они основа .
 технический результат - создание новый фторировать углеводород , обладать высокий совместимость с водородсодержать галогенировать углеводород и хороший стойкость , который растворяться в обычный органический растворитель и мочь быть применимый в качество смазок и масло для холодильный машина .
 12 с . и

способ получение фторировать олефин  . 
 1 .
 способ получение дигидрофторолефин , иметь формула rch=chc2f5 , включать взаимодействие фторированный олефин формула rch = chf , где r представлять себя перфторировать алкильный группа , иметь от один до десять атом углерод , и указанный алкильный группа иметь или н-алкильную цепь , втор-алкильную цепь или изоалкильный цепь , в жидкий фаза с тетрафторэтилен в присутствие в качество катализатор кислота льюис ; удаление кислота льюис , являть катализатор ; и выделение дигидрофторолефин .
 2 .
 способ по п.1 , дополнительно включать стадия водный промывка после реакция для полный удаление остаточный кислота льюис .
 3 .
 способ по п.2 , где стадия водный промывка включать промывка продукт водный фосфатный буферный раствор .
 4 . способ по п.1 , где реакция проводить при температура от примерно -10 до примерно 30°с .
 5 .
 способ по п.4 , где реакция проводить при температура от примерно -5 до примерно 25°с .
 6 .
 способ по п.1 , где тетрафтор

способ каталитический получение частично галогенированный пропанов и частично фторировать бутан ( вариант )  . 
 изобретение относиться к способ получение частично галогенированный прапывать или частично фторированный бутан , включать : взаимодействие частично фторированный метан , который выбирать из группа , состоять из ch2f2 к ch3f , с фторолефин , который выбирать из группа , состоять из cf2=cf2 , clfc=cf2 и cf3cf2=cf2 , в присутствие катализатор - модифицированный хлорида алюминий , где модифицировать хлорид алюминий представлять формула alcl3-xfx , в который х равный от примерно 1,0 до примерно 2,8 или модифицированный бромида алюминий , где модифицировать бромид алюминий представлять формула albr3-yfy , в который у равный от примерно 2,7 до примерно 2,95 .
 причем в процесс сначала вводить в реакционный сосуд катализатор , потом подавать в указанный реакционный сосуд частично фторированный метан .
 далее , можно осуществлять взаимодействие указанный частично фторированный метан 